In [1]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Activation
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
import importlib

import utilis

Using TensorFlow backend.


In [2]:
# Check that you are running GPU's
utilis.GPU_checker()





You are runnning an instance with 4 GPU's


In [4]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7fcfc0266ba8>


In [4]:
sys.path.append('..')

In [5]:
# generator config
W_HYP = True

In [8]:
%%time
from packages import generator
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator(data_dir='',  dataset='train_new', w_hyp=W_HYP)
validation_generator = generator.Keras_DataGenerator(data_dir='', dataset='valid_new', w_hyp= W_HYP)

# of batches:  5888.0
Generating examples from a set of 376832 examples 

# of batches:  1600.0
Generating examples from a set of 102400 examples 

CPU times: user 5.66 ms, sys: 195 µs, total: 5.86 ms
Wall time: 3.92 ms


In [7]:
#Constants
# ARE YOU LOADINNG MODE?
VOCAB_SIZE = 1254
EMBEDDING_DIM = 512


# constnats
INPUT_LENGTH = 3000 if W_HYP else 1000
print(INPUT_LENGTH)


# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

3000

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3000, 512)         642048    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 3000, 512)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
____________________

In [13]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
#overwriting model
# model = loaded_model

In [10]:
%%time
#try and make it run until 9 am GMT+1
n_epochs = 2
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            verbose=1,
                            use_multiprocessing=True,
                            epochs=n_epochs)

Epoch 1/2
1000/1000 [==============================] - 1959s 2s/step - loss: 0.8245 - acc: 0.7337 - val_loss: 4.2367 - val_acc: 0.2459
Epoch 2/2
1000/1000 [==============================] - 1938s 2s/step - loss: 0.3236 - acc: 0.9018 - val_loss: 5.4069 - val_acc: 0.2460
CPU times: user 24min 52s, sys: 6min 56s, total: 31min 49s
Wall time: 1h 4min 57s


In [12]:
# FOR SAVING MODEL
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [13]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "forjeff2"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [14]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

FileNotFoundError: [Errno 2] No such file or directory: 'training_logs/forjeff2.csv'